# Scrap individual data from the website
This code opens the data files for the male and female runners, creates a singel data frama and merges it into one file.
For each runner it goes to the website in the Link, extracts the runners times, and save that info into individual files. Another script will then merger that info into a single file.

In [1]:
import pandas as pd
import numpy as np
from lxml import html
import requests

# data in Data/
#Male runners
m_data0 = pd.read_csv("Data/ChicagoMarathon_Male00.csv") #1-1105, 1000 runners
m_data1 = pd.read_csv("Data/ChicagoMarathon_Male01.csv") #

#female runners
f_data0 = pd.read_csv("Data/ChicagoMarathon_Female00.csv") #
f_data1 = pd.read_csv("Data/ChicagoMarathon_Female01.csv") #

In [2]:
#Merge the data and create a gender label
mdata = [m_data0, m_data1]
fdata = [f_data0, f_data1]

male_data = pd.concat(mdata,ignore_index=True)
male_data['Gender'] = 'Male'

female_data = pd.concat(fdata,ignore_index=True)
female_data['Gender'] = 'Female'

In [3]:
#merge female and male datasets
mydata = [male_data, female_data]
data = pd.concat(mydata,ignore_index=True)
outfile = 'Data/Runner_Data.csv'
data.to_csv(outfile, sep=',')


In [4]:
data.head(1)

,Place Overall,Place Gender,Place Division,Name (CTZ),Link,"City, State",BIB,Division,Age,HALF,Finish,Gender
0,1,1,1.0,"» Kirui, Abel (KEN)",?content=detail&fpid=list&pid=list&idp=9999991...,NaN,3,30-34,34.0,01:06:51,02:11:23,Male


In [7]:
#Add some more info to the runners
pace_data = ['t05k','t10k','t15k','t20k','tHalf','t25k','t30k','t35k','t40k','tFinal']
mperk_data = ['mk05','mk10','mk15','mk20','mkHalf','mk25','mk30','mk35','mk40','mkFinal']
kmh_data = ['kmh05','kmh10','kmh15','kmh20','kmhHalf','kmh25','kmh30','kmh35','kmh40','kmhFinal']
run_data = ['start_time']
for i in range(len(pace_data)):
    run_data.append(pace_data[i])
for i in range(len(mperk_data)):
    run_data.append(mperk_data[i])
for i in range(len(kmh_data)):
    run_data.append(kmh_data[i])

In [8]:
#rdata = pd.Series(run_data, index=index)
rdata = pd.DataFrame(columns =run_data,index=['0'])

In [9]:
rdata.head()

,start_time,t05k,t10k,t15k,t20k,tHalf,t25k,t30k,t35k,t40k,...,kmh05,kmh10,kmh15,kmh20,kmhHalf,kmh25,kmh30,kmh35,kmh40,kmhFinal
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
data.head()

,Place Overall,Place Gender,Place Division,Name (CTZ),Link,"City, State",BIB,Division,Age,HALF,Finish,Gender
0,1,1,1.0,"» Kirui, Abel (KEN)",?content=detail&fpid=list&pid=list&idp=9999991...,NaN,3,30-34,34.0,01:06:51,02:11:23,Male
1,2,2,1.0,"» Chumba, Dickson (KEN)",?content=detail&fpid=list&pid=list&idp=9999991...,NaN,1,25-29,29.0,01:06:51,02:11:26,Male
2,3,3,1.0,"» Kipketer, Gideon (KEN)",?content=detail&fpid=list&pid=list&idp=9999991...,NaN,7,20-24,23.0,01:06:51,02:12:20,Male
3,4,4,2.0,"» Lonyangata, Paul (KEN)",?content=detail&fpid=list&pid=list&idp=0000001...,NaN,6,20-24,24.0,01:06:51,02:13:17,Male
4,5,5,2.0,"» Sambu, Stephen (KEN)",?content=detail&fpid=list&pid=list&idp=9999991...,NaN,8,25-29,28.0,01:06:51,02:13:35,Male


In [14]:
import time
starttime = time.time()
localtime = starttime
mod = 250
web_base = 'http://results.chicagomarathon.com/2016/'
for nl in range(0,len(data.Link)):

    runner_link = web_base + data.Link[nl]

    page = requests.get(runner_link)
    tree = html.fromstring(page.content)

    stime = tree.xpath('//td[@class="f-starttime_net last"]/text()')
    time_pace = tree.xpath('//td[@class="time"]/text()')
    outfile = 'Data/RunnerData/RunnerBIB_%05d.csv' % data.BIB[nl]
    
    min_km = tree.xpath('//td[@class="min_km"]/text()')    #minutes per mile
    kmh_last = tree.xpath('//td[@class="kmh last"]/text()') #miles per hour

    rdata.start_time[0] = stime[0]
    for i in range(len(pace_data)):
        rdata[pace_data[i]][0] = time_pace[i]
    for i in range(len(mperk_data)):
        rdata[mperk_data[i]][0] = min_km[i]
    for i in range(len(kmh_data)):
        rdata[kmh_data[i]][0] = kmh_last[i]

        
    outfile = 'Data/RunnerData/RunnerBIB_%05d.csv' % data.BIB[nl]
    rdata.to_csv(outfile, sep=',')
    if nl%mod==0:
        prevtime = localtime
        localtime = time.time()
        print 'sec per time %1.3f' %((localtime-prevtime)/mod)
        print 'reading line ', nl
print 'DONE'
print 'total time %1.2f mins' % ((localtime - starttime)/60)

sec per time 0.002
reading line  22750
sec per time 0.275
reading line  23000
sec per time 0.275
reading line  23250
sec per time 0.262
reading line  23500
sec per time 0.264
reading line  23750
sec per time 0.282
reading line  24000
sec per time 0.282
reading line  24250
sec per time 0.273
reading line  24500
sec per time 0.275
reading line  24750
sec per time 0.284
reading line  25000
sec per time 0.275
reading line  25250
sec per time 0.282
reading line  25500
sec per time 0.289
reading line  25750
sec per time 0.272
reading line  26000
sec per time 0.307
reading line  26250
sec per time 0.290
reading line  26500
sec per time 0.279
reading line  26750
sec per time 0.281
reading line  27000
sec per time 0.296
reading line  27250
sec per time 0.285
reading line  27500
sec per time 0.284
reading line  27750
sec per time 0.285
reading line  28000
sec per time 0.279
reading line  28250
sec per time 0.296
reading line  28500
sec per time 0.285
reading line  28750
sec per time 0.281
readin

In [12]:
len(data.Link)

40547